In [17]:
!pip install pandas
!pip install imbalanced-learn
!pip install scikeras[tensorflow]

In [18]:
!chmod +x /kaggle/input/classification/download_cl.sh
!source /kaggle/input/classification/download_cl.sh

chmod: impossível aceder a '/kaggle/input/classification/download_cl.sh': Ficheiro ou pasta inexistente
/bin/bash: linha 1: /kaggle/input/classification/download_cl.sh: Ficheiro ou pasta inexistente


In [19]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as  plt
from imblearn.under_sampling import RandomUnderSampler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [20]:
def filter_data(x,y,color = 1):
    x = x.values
    mask = x[:,64] == color
    filtered_x = x[mask]
    filtered_y = y[mask]
    board = filtered_x[:,0:64].reshape((filtered_x.shape[0],8,8))
    return board,filtered_y

In [21]:
def load_n_datasets(n,foldername):
    all_datasets = []
    filenames = os.listdir(foldername)
    for i in range(n):
        filename = filenames[i]
        if filename.endswith('.csv'):
            dataset = pd.read_csv(os.path.join(foldername, filename))
            all_datasets.append(dataset)
    combined_dataset = pd.concat(all_datasets, ignore_index=True)
    
    X =  combined_dataset.iloc[:,0:-1]
    y =  combined_dataset["Evaluation"].astype(int)
    
    print(type(X))
    print(X.shape)
    
    X,y = filter_data(X,y)
    
    print(type(X))
    print(X.shape)
    
    X = X.reshape(X.shape[0], -1)
    
    print(type(X))
    print(X.shape)
    
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    
    undersampler = RandomUnderSampler()
    X_resampled, y_resampled = undersampler.fit_resample(X, y)
    return X_resampled,y_resampled

In [22]:
def bit_map(X):
    channels = np.zeros((X.shape[0],8, 8, 12))  # 12 channels for 6 types of pieces for each player
    # Generate separate channels for each player
    for player in range(2):  # 0 for white pieces, 1 for black pieces
        if player ==0:
            for piece_type in range(6):  # 6 types of pieces
                piece_mask = X == (piece_type + 1)
                channels[:, :, :, player * 6 + piece_type] = piece_mask.astype(np.float32)
        else:
            for piece_type in range(6):  # 6 types of pieces
                piece_mask = -1*(X == (-piece_type - 1))
                channels[:, :, :, player * 6 + piece_type] = piece_mask.astype(np.float32)
    return channels

In [23]:
n_datasets = 1
X_chess_data,y_chess_data = load_n_datasets(n_datasets,'../classification_data')

#X_chess_data, _, y_chess_data, _ = train_test_split(X_chess_data, y_chess_data, test_size=0.01, random_state=42, stratify=y_chess_data)

print(type(X_chess_data))
print(X_chess_data.shape)

X = np.array(X_chess_data)
y = np.array(y_chess_data)
X = X.reshape((X.shape[0],8,8))
y = to_categorical(y)
X = bit_map(X)

<class 'pandas.core.frame.DataFrame'>
(996771, 69)
<class 'numpy.ndarray'>
(498279, 8, 8)
<class 'numpy.ndarray'>
(498279, 64)
<class 'pandas.core.frame.DataFrame'>
(177021, 64)


In [16]:

print(type(X))
print(X.shape)
print(type(y))
print(y.shape)
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val,X_test,y_val,y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)
#80% training, 10% validation and 10% testing.

<class 'numpy.ndarray'>
(177021, 8, 8, 12)
<class 'numpy.ndarray'>
(177021, 13)


In [9]:
import tensorflow
from keras.models import Sequential 
from keras.layers import Conv2D, Flatten, BatchNormalization, Dense, Dropout
from keras.activations import elu
from keras.optimizers import SGD,Adam

tensorflow.random.set_seed(101)

In [10]:
def create_model_mlp1(input_shape,num_classes):
    mlp_model = Sequential()

    # Add hidden layers with ReLU activation and dropout
    mlp_model.add(Dense(1048, activation='relu', input_shape=input_shape))
    mlp_model.add(Dropout(0.2))
    mlp_model.add(Dense(500, activation='relu'))
    mlp_model.add(Dropout(0.2))
    mlp_model.add(Dense(50, activation='relu'))
    mlp_model.add(Dropout(0.2))

    # Layer to ensure the output matches the labels in shape
    mlp_model.add(Flatten())
    
    # Add output layer with softmax activation
    mlp_model.add(Dense(num_classes, activation='softmax'))

    # Compile model with Adam optimizer
    adam = Adam(learning_rate=0.001, beta_1=0.90, beta_2=0.99, epsilon=1e-8)
    mlp_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    mlp_model.summary()
    return mlp_model

In [11]:
def create_model_cnn(input_shape,num_classes):
    cnn_model = Sequential()

    # Add convolutional layers with ELU activation and batch normalization
    cnn_model.add(Conv2D(20, (5, 5), activation=elu, input_shape=input_shape, padding='same'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(Dropout(0.3))
    
    cnn_model.add(Conv2D(50, (3, 3), activation=elu, padding='same'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(Dropout(0.3))
    
    # Add fully connected layer
    cnn_model.add(Flatten())
    cnn_model.add(Dense(500, activation=elu))
    cnn_model.add(Dropout(0.3))
    
    # Add output layer with softmax activation
    cnn_model.add(Dense(num_classes, activation='softmax'))

    # Compile model with SGD optimizer
    sgd = SGD(learning_rate=0.01, epsilon=1e-8, nesterov=False)
    cnn_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    cnn_model.summary()
    return cnn_model

In [12]:
def create_model_mlp2(input_shape,num_classes):
    mlp_model = Sequential()

    # Add hidden layers with ReLU activation and dropout
    mlp_model.add(Dense(2048, activation='elu', input_shape=input_shape))
    mlp_model.add(Dropout(0.2))
    mlp_model.add(Dense(2048, activation='elu'))
    mlp_model.add(Dropout(0.2))
    mlp_model.add(Dense(1050, activation='elu'))
    mlp_model.add(Dropout(0.2))

    # Layer to ensure the output matches the labels in shape
    mlp_model.add(Flatten())
    
    # Add output layer with softmax activation
    mlp_model.add(Dense(num_classes, activation='softmax'))

    # Compile model with Adam optimizer
    adam = Adam(learning_rate=0.001, beta_1=0.90, beta_2=0.99, epsilon=1e-8)
    mlp_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    mlp_model.summary()
    return mlp_model

In [13]:
def run_model(X_train,y_train,X_val,y_val,model_params,batch_size,epochs):

    #model = create_model(
    #                     activation=model_params['activation'],
    #                     n_layers=model_params['n_layers'],
    #                     n_neurons=model_params['n_neurons'],
    #                     dropout=model_params['dropout'],
    #                     l2=model_params['l2']
    #                     )
    
    input_shape = X_train.shape[1:]
    num_classes = 13
    print(input_shape)
    model = create_model_mlp1(input_shape,num_classes)
    #model = create_model_cnn(input_shape,num_classes)
    #model = create_model_mlp2(input_shape,num_classes)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    return history,model

In [14]:
model_params = {
    'activation' : 'elu',
    'n_layers' : 3,
    'n_neurons' : 300,
    'dropout' : 0.4,
    'l2' : 0.01
}
batch_size = 128
epochs = 30

history,model = run_model(X_train,y_train,X_val,y_val,model_params,batch_size,epochs)

(8, 8, 12)


2024-04-14 20:29:47.191138: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8, 8, 1048)        13624     
                                                                 
 dropout (Dropout)           (None, 8, 8, 1048)        0         
                                                                 
 dense_1 (Dense)             (None, 8, 8, 500)         524500    
                                                                 
 dropout_1 (Dropout)         (None, 8, 8, 500)         0         
                                                                 
 dense_2 (Dense)             (None, 8, 8, 50)          25050     
                                                                 
 dropout_2 (Dropout)         (None, 8, 8, 50)          0         
                                                                 
 flatten (Flatten)           (None, 3200)              0

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score

loss,acc = model.evaluate(X_test,y_test)

print("Loss:",loss)
print("Accuracy:",acc)

In [ ]:
epochs_range = range(1, epochs + 1)

plt.plot(epochs_range, history.history['loss'], label='Training Loss')
plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()